In [3]:
import cv2
import math
import numpy as np
import time

In [21]:
image = cv2.imread('citra16.jpg')


In [5]:
def gray_convert(image):
    rgb_weights = [0.2989, 0.5870, 0.1140]
    grayscale_image = np.dot(image[...,:3], rgb_weights)
    gray_img = grayscale_image.astype(np.uint8)
    return gray_img

In [6]:
img = gray_convert(image)

In [7]:
array_gray = np.array(img)
print(array_gray)

[[121 120 119 ... 124 117 108]
 [124 122 120 ... 126 119 111]
 [124 122 120 ... 128 122 115]
 ...
 [203 196 184 ... 183 164 120]
 [199 194 189 ... 174 176 165]
 [192 191 191 ... 169 172 172]]


In [8]:
cv2.imwrite('grayscale.png', img)
# cv2.imshow('grayscale', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
def summed_area_table(img):

    table = np.zeros_like(img).astype(int)

    for baris in range(img.shape[0]):
        for kolom in range(img.shape[1]):

            if (baris > 0) and (kolom > 0):
                table[baris, kolom] = (img[baris, kolom] +
                                   table[baris, kolom - 1] +
                                   table[baris - 1, kolom] -
                                   table[baris - 1, kolom - 1])
            elif baris > 0:   
                table[baris, kolom] = img[baris, kolom] + table[baris - 1, kolom]
            elif kolom > 0:
                table[baris, kolom] = img[baris, kolom] + table[baris, kolom - 1]
            else:
                table[baris, kolom] = img[baris, kolom]

    return table

In [10]:
summed_area_table(img)

array([[     121,      241,      360, ...,    39100,    39217,    39325],
       [     245,      487,      726, ...,    78313,    78549,    78768],
       [     369,      733,     1092, ...,   117507,   117865,   118199],
       ...,
       [   48476,    96860,   145295, ..., 15211019, 15258821, 15306280],
       [   48675,    97253,   145877, ..., 15258893, 15306871, 15354495],
       [   48867,    97636,   146451, ..., 15306983, 15355133, 15402929]])

In [14]:
def thresholdIntegral(inputMat,s,T = 0.15):
    outputMat=np.zeros(inputMat.shape) #output adaptif threshold 
    baris = inputMat.shape[0]
    kolom = inputMat.shape[1]
    S = int(max(baris, kolom) / 8) #kernel dari total piksel citra

    s2 = int(S / 2)

    # penentuan posisi kernel
    for i in range(baris):
        y1 = i - s2
        y2 = i + s2

        if (y1 < 0) :
            y1 = 0
        if (y2 >= baris):
            y2 = baris - 1

        for j in range(kolom):
            x1 = j - s2
            x2 = j + s2

            if (x1 < 0) :
                x1 = 0
            if (x2 >= kolom):
                x2 = kolom - 1
            count = (x2 - x1)*(y2 - y1)
            
            # rumus rata-rata integral
            sum=s[y2][x2]-s[y2][x1]-s[y1][x2]+s[y1][x1]
            
            # pengujian nilai threshold
            if ((int)(inputMat[i][j] * count) <= (int)(sum*(1.0 - T))):
                outputMat[i][j] = 0
            else:
                outputMat[i][j] = 255
    return outputMat

In [19]:
if __name__ == '__main__':
    ratio=1
    image = cv2.imdecode(np.fromfile('citra16.jpg', dtype=np.uint8), 0)
    img = cv2.resize(image, (int(image.shape[1] / ratio), int(image.shape[0] / ratio)), cv2.INTER_NEAREST)

    time_start = time.time()
    roii = summed_area_table(img)
    time_end = time.time()
    print('integral cost', time_end - time_start)

    # menampilkan threshold
#     for j in range(1):
    thresh = thresholdIntegral(img, roii)
    time_end = time.time()
    print('totally cost', time_end - time_start)
    print(thresh)
#     cv2.namedWindow('integral threshold',0)
#     cv2.imshow('integral threshold',thresh)
    cv2.imwrite('outCitra16.png', np.uint8(thresh))

#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

integral cost 0.3374178409576416
totally cost 1.0583269596099854
[[255. 255. 255. ... 255. 255.   0.]
 [255. 255. 255. ... 255. 255. 255.]
 [255. 255. 255. ... 255. 255. 255.]
 ...
 [255. 255. 255. ... 255. 255. 255.]
 [255. 255. 255. ... 255. 255. 255.]
 [255. 255. 255. ... 255. 255. 255.]]
